In [1]:
require './GenericConvolution.rb'
require './WaveletFilters.rb'

true

In [2]:
openmp = false

def Kinetic1d(filter, optims=GenericOptimization::new)
  conv_operation = GenericConvolutionOperator1d::new(filter, :kinetic => :inplace, :ld => true, 
    :narr => true, :a_x => true,:a_y=>true, :a => true, :dot_in => true)
  conv_operation.optimize(optims) if optims

  p, subops = conv_operation.procedure

  kernel = BOAST::CKernel::new

  print_header

  subops.each_value { |op|
    BOAST::pr op
    puts "chosen:"+ op.name
  }
  BOAST::pr p

  kernel.procedure = p
  kernel.cost_function = lambda { |*args| conv_operation.cost(*args) }
  return kernel
end


:Kinetic1d

In [3]:
n1 = 32
n2 = 32
n3 = 32

optims = GenericOptimization::new(:repeat => 1,
  :unroll_range => [1,1], :dimensions => [n1,n2,n3],:openmp => openmp, :mod_arr_test => false,
  :tt_arr_test => false)
conv_filter = ConvolutionFilter::new('sym8_d2',SYM8_D2,14)
5 #to output

5

In [4]:
kkin8 = Kinetic1d( conv_filter, optims )
5

d_sym8_d2_p_10_dotin_ld_u1_v1_1_t_f_t: 0.82624 ms 2.300232378001549 GFlops
d_sym8_d2_p_10_a_dotin_ld_u1_v1_1_t_f_t: 0.865236 ms 2.196561400588972 GFlops
d_sym8_d2_p_10_ain_dotin_ld_u1_v1_1_t_f_t: 1.057496 ms 1.797211526095607 GFlops
d_sym8_d2_p_10_ay_dotin_ld_u1_v1_1_t_f_t: 1.0513990000000002 ms 1.8076334483863876 GFlops
d_sym8_d2_p_10_a_ain_dotin_ld_u1_v1_1_t_f_t: 1.128382 ms 1.684309037187761 GFlops
d_sym8_d2_p_10_a_ay_dotin_ld_u1_v1_1_t_f_t: 0.850222 ms 2.235350296746026 GFlops
d_sym8_d2_p_10_ain_ay_dotin_ld_u1_v1_1_t_f_t: 0.827425 ms 2.2969380910656554 GFlops
d_sym8_d2_p_10_a_ain_ay_dotin_ld_u1_v1_1_t_f_t: 1.085218 ms 1.7513015818019975 GFlops
d_sym8_d2_p_01_dotin_ld_u1_v1_0_t_f_t: 1.0676260000000002 ms 1.7801589695267814 GFlops
d_sym8_d2_p_01_a_dotin_ld_u1_v1_0_t_f_t: 1.216889 ms 1.5618055549848835 GFlops
d_sym8_d2_p_01_ain_dotin_ld_u1_v1_0_t_f_t: 1.0923 ms 1.7399469010345143 GFlops
d_sym8_d2_p_01_ay_dotin_ld_u1_v1_0_t_f_t: 1.2047310000000002 ms 1.577567108342028 GFlops
d_sym8_d2_

5

SUBROUTINE d_sym8_d2_p_10_dotin_ld_u1_v1_1_t_f_t(n, ndat, nx, ny, x, y, dot_in)
  integer, parameter :: wp=kind(1.0d0)
  integer(kind=4), parameter :: lowfil = -14
  integer(kind=4), parameter :: upfil = 14
  integer(kind=4), intent(in) :: n
  integer(kind=4), intent(in) :: ndat
  integer(kind=4), intent(in) :: nx
  integer(kind=4), intent(in) :: ny
  real(kind=8), intent(in), dimension(0:nx - (1), 0:ndat - (1)) :: x
  real(kind=8), intent(out), dimension(0:ny - (1), 0:ndat - (1)) :: y
  real(kind=8), intent(out) :: dot_in
  real(kind=8), parameter, dimension(-14:14) :: sym8_d2_fil = reshape((/ &
-6.924474940639200152025730585882115968579663856946045039077208122e-18_wp, &
2.708004936263194382698566890376475760883688432060067551662628469e-13_wp, &
-5.813879830282540547959250667464887365484326600877085719823602219e-11_wp, &
-1.058570554967414703734941322870242653300359680996853899470022889e-8_wp, &
-3.723076304736927584879149697304408803274215476409064356088861068e-7_wp, &
2.090423495292

In [5]:
#puts kkin8
kkin8.build(:openmp => openmp)
5

5

In [6]:
require './Kinetic.rb'

true

In [7]:
kref = kinetic_per_ref_optim
5

5

In [8]:
kref.build(:openmp => openmp)
5

5

In [23]:
input = ANArray.float(32,n1,n2,n3).random! #fill!(1.0)
output_ref = ANArray.float(32,n1,n2,n3)
output = ANArray.float(32,n1,n2,n3)
hgrid = NArray.float(3)
hgrid[0] = 0.5
hgrid[1] = 0.6
hgrid[2] = 0.7
c=0.5

0.5

In [24]:
stats = kref.run(n1-1, n2-1, n3-1, hgrid, input, output_ref, c)

{:duration=>0.0013762700000000002, :energy=>{:"0.package-0"=>0.01831, :"0.core"=>0.014098999999999999, :"0.dram"=>0.001648}}

In [25]:
puts input[0]
output_ref[0]

0.7036452163103972


-1.0751685837039395

In [26]:
ndims=NArray.int(3)
ndims[0]=n1
ndims[1]=n2
ndims[2]=n3
bc = BC::PERIODIC
c=0.5
eval=0.0
scal=NArray.float(3)
(0..2).each{ |ind| scal[ind] = -0.5 / (hgrid[ind]*hgrid[ind])   }

0..2

In [27]:
kkin8

SUBROUTINE d_sym8_d2_p_10_dotin_ld_u1_v1_1_t_f_t(n, ndat, nx, ny, x, y&
&, dot_in)
  integer, parameter :: wp=kind(1.0d0)
  integer(kind=4), parameter :: lowfil = -14
  integer(kind=4), parameter :: upfil = 14
  integer(kind=4), intent(in) :: n
  integer(kind=4), intent(in) :: ndat
  integer(kind=4), intent(in) :: nx
  integer(kind=4), intent(in) :: ny
  real(kind=8), intent(in), dimension(0:nx - (1), 0:ndat - (1)) :: x
  real(kind=8), intent(out), dimension(0:ny - (1), 0:ndat - (1)) :: y
  real(kind=8), intent(out) :: dot_in
  real(kind=8), parameter, dimension(-14:14) :: sym8_d2_fil = reshape(&
&(/ &
-6.924474940639200152025730585882115968579663856946045039077208122e-18&
&_wp, &
2.708004936263194382698566890376475760883688432060067551662628469e-13_&
&wp, &
-5.813879830282540547959250667464887365484326600877085719823602219e-11&
&_wp, &
-1.058570554967414703734941322870242653300359680996853899470022889e-8_&
&wp, &
-3.723076304736927584879149697304408803274215476409064356088861068e-7_&


1D operation in the x direction

In [28]:
stat=kkin8.run(3,0,ndims,bc,ndims,ndims,1,input,output,scal[0],c,0.0,eval) # out= scal[0] * d2x [in] + c * in + 0.0 * out

{:reference_return=>{:dot_in=>19378.864632793782}, :duration=>0.000976675, :energy=>{:"0.package-0"=>0.016968, :"0.core"=>0.012756, :"0.dram"=>0.001526}}

In [29]:
eval=stat[:reference_return][:dot_in]
puts eval

19378.864632793782


y and z direction

In [30]:
kkin8.run(3,1,ndims,bc,ndims,ndims,1,input,output,scal[1],0.0,1.0,eval) # out= scal[1]* d2y[in]  + 1.0 * out
eval+=stat[:reference_return][:dot_in]
kkin8.run(3,2,ndims,bc,ndims,ndims,1,input,output,scal[2],0.0,1.0,eval) # out= scal[2]* d2z[in]  + 1.0 * out
eval+=stat[:reference_return][:dot_in]


58136.59389838135

In [31]:
puts input[0]
output[0]

0.7036452163103972


-1.075168583703939

In [33]:
eval*hgrid[0]*hgrid[1]*hgrid[2]

12208.684718660083

In [34]:
epsilon=1.0e-13
diff = (output_ref - output).abs
diff.each { |elem|
  raise "Warning: residue too big: #{elem}" if elem > epsilon
}
puts 'ok'

ok


In [36]:
File.open('Kinetic8.f90','w'){ | f |
  f.write(kkin8.to_s)
  }

915430